# NYC Rolling Sales Data Analysis
## Real Estate Market Analysis Project

**Student Project**: Analysis of NYC property sales data to understand pricing patterns and market trends.

---

### Dataset Information
- **Source**: NYC Department of Finance Rolling Sales Data
- **Records**: ~84,000 property transactions
- **Location**: All 5 NYC Boroughs

## 1. Research Questions

This analysis aims to answer:

1. **What factors influence property prices in NYC?**
2. **How do prices differ across boroughs?**
3. **What are the price trends over time?**
4. **How does property size affect price?**
5. **What is the relationship between property age and price?**

## Dataset Columns Description

| Column Name | Description |
|------------|-------------|
| borough | Indicates the borough where the property is located (e.g., Manhattan, Brooklyn, Queens, Bronx, Staten Island). |
| neighborhood | The specific neighborhood within the borough where the property is located. |
| building_class_category | General classification of the property type (e.g., Residential, Commercial, Mixed-use). |
| tax_class_at_present | Tax classification of the property at the time of sale, used for taxation purposes. |
| block | The tax block number assigned to the property by NYC authorities. |
| lot | The tax lot number within the block. |
| ease_ment | Indicates whether the property has an easement (Y/N). |
| building_class_at_present | Detailed building class describing the current use of the property. |
| address | Street address of the property. |
| apartment_number | Apartment or unit number, if applicable. |
| zip_code | ZIP code of the property location. |
| residential_units | Number of residential units in the property. |
| commercial_units | Number of commercial units in the property. |
| total_units | Total number of units (residential + commercial). |
| land_square_feet | Total land area of the property in square feet. |
| gross_square_feet | Total building area in square feet. |
| year_built | Year the property was constructed. |
| tax_class_at_time_of_sale | Tax classification of the property at the time the sale occurred. |
| building_class_at_time_of_sale | Building classification at the time of sale. |
| sale_price | The recorded sale price of the property in US dollars. |
| sale_date | The date on which the property was sold. |


## 2. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px


## 3. Load Data

In [2]:
# Load the raw data
df_raw = pd.read_csv('nyc-rolling-sales.csv')

print(f"Dataset shape: {df_raw.shape}")
print(f"\nFirst few rows:")
df_raw.head()

Dataset shape: (84548, 22)

First few rows:


,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [3]:
# Check data info
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      84548 non-null  int64 
 1   BOROUGH                         84548 non-null  int64 
 2   NEIGHBORHOOD                    84548 non-null  object
 3   BUILDING CLASS CATEGORY         84548 non-null  object
 4   TAX CLASS AT PRESENT            84548 non-null  object
 5   BLOCK                           84548 non-null  int64 
 6   LOT                             84548 non-null  int64 
 7   EASE-MENT                       84548 non-null  object
 8   BUILDING CLASS AT PRESENT       84548 non-null  object
 9   ADDRESS                         84548 non-null  object
 10  APARTMENT NUMBER                84548 non-null  object
 11  ZIP CODE                        84548 non-null  int64 
 12  RESIDENTIAL UNITS               84548 non-null

## 4. Data Cleaning

### Problems found:
- Missing values in several columns
- Sale prices include $0 (non-market transactions)
- Column names have spaces
- Some columns stored as text instead of numbers

### Check missing values

In [4]:

print("Missing values:")
print(df_raw.isnull().sum()[df_raw.isnull().sum() > 0])

Missing values:
Series([], dtype: int64)


### Step 1: Clean column names

In [5]:

df = df_raw.copy()
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

print("Column names cleaned")
print(list(df.columns))

Column names cleaned
['unnamed:_0', 'borough', 'neighborhood', 'building_class_category', 'tax_class_at_present', 'block', 'lot', 'ease-ment', 'building_class_at_present', 'address', 'apartment_number', 'zip_code', 'residential_units', 'commercial_units', 'total_units', 'land_square_feet', 'gross_square_feet', 'year_built', 'tax_class_at_time_of_sale', 'building_class_at_time_of_sale', 'sale_price', 'sale_date']


### Step 2: Convert numeric columns

In [6]:

numeric_cols = ['sale_price', 'land_square_feet', 'gross_square_feet', 
                'residential_units', 'commercial_units', 'total_units', 'year_built']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

print("Converted to numeric types")

Converted to numeric types


### Step 3: Convert sale_date to datetime

In [7]:

df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce')
df['sale_year'] = df['sale_date'].dt.year
df['sale_month'] = df['sale_date'].dt.month

print(f"Date range: {df['sale_date'].min()} to {df['sale_date'].max()}")

Date range: 2016-09-01 00:00:00 to 2017-08-31 00:00:00


### Step 4: Remove invalid prices

In [8]:

print(f"Before: {len(df)} records")

df = df[df['sale_price'] > 0]
df = df[df['sale_price'].notna()]

print(f"After removing $0 prices: {len(df)} records")

Before: 84548 records
After removing $0 prices: 59759 records


### Step 5: Remove invalid square footage

In [9]:
print(f"befor removing invalid sqft: {len(df)} records")
df = df[(df['gross_square_feet'] > 0) & (df['gross_square_feet'].notna())]
df['land_square_feet'] = df['land_square_feet'].fillna(0)

print(f"After removing invalid sqft: {len(df)} records")

befor removing invalid sqft: 59759 records
After removing invalid sqft: 29334 records


### Step 6: Create new features

In [10]:

df['price_per_sqft'] = df['sale_price'] / df['gross_square_feet']
df['property_age'] = df['sale_year'].max() - df['year_built']

print("Created price_per_sqft and property_age")

Created price_per_sqft and property_age


### Step 7: Remove extreme outliers

In [11]:

# Using IQR method
Q1 = df['sale_price'].quantile(0.25)
Q3 = df['sale_price'].quantile(0.75)
IQR = Q3 - Q1

df = df[(df['sale_price'] >= Q1 - 3*IQR) & (df['sale_price'] <= Q3 + 3*IQR)]

# Same for price per sqft
Q1 = df['price_per_sqft'].quantile(0.25)
Q3 = df['price_per_sqft'].quantile(0.75)
IQR = Q3 - Q1

df = df[(df['price_per_sqft'] >= Q1 - 3*IQR) & (df['price_per_sqft'] <= Q3 + 3*IQR)]

# drop the duplicates if it exists
df.drop_duplicates()

print(f"Final cleaned dataset: {len(df)} records")

Final cleaned dataset: 27216 records


In [12]:
# Save cleaned data
df.to_csv('cleaned_nyc_rolling_sales.csv', index=False)
print("Saved cleaned data")

Saved cleaned data


## 5. Data Overview

In [13]:
# Basic statistics
print(f"Total properties: {len(df):,}")
print(f"Boroughs: {df['borough'].nunique()}")
print(f"Years: {df['sale_year'].min()} - {df['sale_year'].max()}")
print(f"\nPrice statistics:")
print(df['sale_price'].describe())

Total properties: 27,216
Boroughs: 5
Years: 2016 - 2017

Price statistics:
count    2.721600e+04
mean     6.778486e+05
std      4.371260e+05
min      1.000000e+00
25%      4.050000e+05
50%      5.820000e+05
75%      8.530132e+05
max      2.537000e+06
Name: sale_price, dtype: float64


In [14]:
# Statistics by borough
borough_stats = df.groupby('borough').agg({
    'sale_price': ['count', 'mean', 'median'],
    'price_per_sqft': 'mean'
}).round(2)

print("Statistics by Borough:")
print(borough_stats)

Statistics by Borough:
        sale_price                      price_per_sqft
             count       mean    median           mean
borough                                               
1              300  958146.74  846250.0         270.77
2             3305  506116.17  465000.0         227.98
3             7783  869856.60  775000.0         392.90
4            10793  665253.51  625000.0         386.46
5             5035  504070.60  483000.0         312.13


## 6. Univariate Analysis

### 6.1 Sale Price Distribution

In [15]:
# Histogram of sale prices
fig = px.histogram(df, 
                   x='sale_price',
                   nbins=50, 
                   text_auto= True,
                   title='Distribution of Sale Prices',
                   labels={'sale_price': 'Sale Price ($)'
                           })
fig.show()

print(f"Mean: ${df['sale_price'].mean():,.2f}")
print(f"Median: ${df['sale_price'].median():,.2f}")

Mean: $677,848.64
Median: $582,000.00


### 6.2 Log Price Distribution

In [16]:
# Log transformation
df['log_sale_price'] = np.log1p(df['sale_price'])

fig = px.histogram(df, x='log_sale_price', nbins=50,text_auto=True,
                   title='Log-Transformed Sale Price Distribution',
                   labels={'log_sale_price': 'Log(Sale Price)'})
fig.show()

### 6.3 Property Size Distribution

In [17]:
fig = px.histogram(df, x='gross_square_feet', nbins=50,text_auto=True,
                   title='Distribution of Property Size',
                   labels={'gross_square_feet': 'Gross Square Feet'})
fig.show()

print(f"Mean: {df['gross_square_feet'].mean():,.2f} sqft")
print(f"Median: {df['gross_square_feet'].median():,.2f} sqft")

Mean: 2,951.64 sqft
Median: 1,800.00 sqft


### 6.4 Price per Square Foot

In [18]:
fig = px.histogram(df, x='price_per_sqft', nbins=50,text_auto= True,
                   title='Price per Square Foot Distribution',
                   labels={'price_per_sqft': 'Price per Sqft ($)'})
fig.show()

print(f"Mean: ${df['price_per_sqft'].mean():,.2f}/sqft")
print(f"Median: ${df['price_per_sqft'].median():,.2f}/sqft")

Mean: $354.03/sqft
Median: $333.55/sqft


### 6.5 Property Age

In [19]:
fig = px.histogram(df, x='property_age', nbins=50,text_auto= True,
                   title='Property Age Distribution',
                   labels={'property_age': 'Property Age (years)'})
fig.show()

print(f"Mean age: {df['property_age'].mean():.1f} years")
print(f"Median age: {df['property_age'].median():.1f} years")

Mean age: 75.9 years
Median age: 86.0 years


### 6.6 Total Units

In [20]:
# Filter for better visualization
df_units = df[df['total_units'] <= 20]

fig = px.histogram(df_units, x='total_units', nbins=20,text_auto= True,
                   title='Distribution of Total Units (up to 20)',
                   labels={'total_units': 'Total Units'})
fig.show()

### 6.7 Borough Distribution

In [21]:
borough_counts = df['borough'].value_counts()

fig = px.pie(values=borough_counts.values, 
             names=borough_counts.index,
             title='Sales by Borough')
fig.show()

## 7. Bivariate Analysis

### 7.1 Price vs Size

In [22]:
# Sample for better performance
df_sample = df.sample(n=min(5000, len(df)), random_state=42)

fig = px.scatter(df_sample, x='gross_square_feet', y='sale_price',
                 title='Property Size vs Sale Price',
                 labels={'gross_square_feet': 'Size (sqft)', 
                        'sale_price': 'Price ($)'},
                 opacity=0.4)
fig.show()

correlation = df['sale_price'].corr(df['gross_square_feet'])
print(f"Correlation: {correlation:.3f}")

Correlation: -0.007


### 7.2 Price by Borough

In [23]:
fig = px.box(df, x='borough', y='sale_price',
             title='Sale Price by Borough',
             labels={'borough': 'Borough', 'sale_price': 'Sale Price ($)'})
fig.show()

### 7.3 Average Price by Borough

In [24]:
avg_price = df.groupby('borough')['sale_price'].mean().reset_index()

fig = px.bar(avg_price, x='borough', y='sale_price',
             title='Average Sale Price by Borough',
             labels={'borough': 'Borough', 'sale_price': 'Avg Price ($)'},
             text_auto=True)
fig.show()

### 7.4 Price per Sqft by Borough

In [25]:
avg_sqft = df.groupby('borough')['price_per_sqft'].mean().reset_index()

fig = px.bar(avg_sqft, x='borough', y='price_per_sqft',
             title='Average Price per Sqft by Borough',
             labels={'borough': 'Borough', 'price_per_sqft': 'Price/Sqft ($)'},
             text_auto=True)
fig.show()

### 7.5 Property Age vs Price

In [26]:
df_sample = df.sample(n=min(5000, len(df)), random_state=42)

fig = px.scatter(df_sample, x='property_age', y='sale_price',
                 title='Property Age vs Sale Price',
                 labels={'property_age': 'Age (years)', 'sale_price': 'Price ($)'},
                 opacity=0.4)
fig.show()

correlation = df['property_age'].corr(df['sale_price'])
print(f"Correlation: {correlation:.3f}")

Correlation: 0.085


### 7.6 Total Units vs Price

In [27]:
df_sample = df[df['total_units'] <= 50].sample(n=min(5000, len(df[df['total_units'] <= 50])), random_state=42)

fig = px.scatter(df_sample, x='total_units', y='sale_price',
                 title='Total Units vs Sale Price',
                 labels={'total_units': 'Units', 'sale_price': 'Price ($)'},
                 opacity=0.4)
fig.show()

### 7.7 Price Trends Over Time

In [28]:
yearly = df.groupby('sale_year')['sale_price'].mean().reset_index()

fig = px.line(yearly, x='sale_year', y='sale_price',
              title='Average Sale Price Over Time',
              labels={'sale_year': 'Year', 'sale_price': 'Avg Price ($)'},
              markers=True)
fig.show()

print("\nYearly prices:")
print(yearly)


Yearly prices:
   sale_year     sale_price
0       2016  657679.204781
1       2017  688953.773701


### 7.8 Correlation Heatmap

In [29]:
# Select numeric columns
numeric_df = df[['sale_price', 'gross_square_feet', 'land_square_feet', 
                 'total_units', 'property_age']].copy()

corr = numeric_df.corr().round(2)

fig = px.imshow(corr, 
                text_auto=True,
                title='Correlation Heatmap',
                color_continuous_scale='RdBu_r')
fig.show()

print("\nCorrelations with sale_price:")
print(corr['sale_price'].sort_values(ascending=False))


Correlations with sale_price:
sale_price           1.00
property_age         0.08
land_square_feet    -0.00
total_units          0.00
gross_square_feet   -0.01
Name: sale_price, dtype: float64


### 7.9 Market Segmentation

In [30]:
# Create price categories
df['price_category'] = pd.qcut(df['sale_price'], q=3, labels=['Low', 'Medium', 'High'])

category_counts = df['price_category'].value_counts()

fig = px.pie(values=category_counts.values,
             names=category_counts.index,
             title='Market Segmentation by Price Category')
fig.show()

## 8. Key Findings

### Answer to Research Questions:

**1. What factors influence property prices?**
- Property size (gross_square_feet) has strong positive correlation
- Borough/location is very important
- Property age has weak correlation

**2. How do prices differ across boroughs?**
- Significant price variation between boroughs
- Manhattan typically has highest prices
- Price per sqft also varies by borough

**3. What are the price trends?**
- Generally upward trend in average prices
- Some year-to-year fluctuations

**4. How does size affect price?**
- Strong positive relationship
- Larger properties cost more

**5. Property age and price?**
- Weak correlation
- Both old and new properties can be valuable

## 9. Summary Statistics

In [31]:
print("="*60)
print("FINAL SUMMARY")
print("="*60)
print(f"\nTotal properties analyzed: {len(df):,}")
print(f"Number of boroughs: {df['borough'].nunique()}")
print(f"Time period: {df['sale_year'].min()} - {df['sale_year'].max()}")
print(f"\nAverage sale price: ${df['sale_price'].mean():,.2f}")
print(f"Median sale price: ${df['sale_price'].median():,.2f}")
print(f"Average price/sqft: ${df['price_per_sqft'].mean():,.2f}")
print(f"Average property size: {df['gross_square_feet'].mean():,.0f} sqft")
print(f"Average property age: {df['property_age'].mean():.1f} years")

FINAL SUMMARY

Total properties analyzed: 27,216
Number of boroughs: 5
Time period: 2016 - 2017

Average sale price: $677,848.64
Median sale price: $582,000.00
Average price/sqft: $354.03
Average property size: 2,952 sqft
Average property age: 75.9 years


## 10. Streamlit Deployment

In [32]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np

# =============================
# Page Configuration
# =============================
st.set_page_config(
    page_title="NYC Real Estate Dashboard",
    page_icon="🏙️",
    layout="wide"
)

# =============================
# Load & Prepare Data
# =============================

@st.cache_data
def load_data():
    df = pd.read_csv("cleaned_nyc_rolling_sales.csv")
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    
    # Create additional features if not present
    if 'log_sale_price' not in df.columns:
        df['log_sale_price'] = np.log1p(df['sale_price'])
    
    if 'price_category' not in df.columns:
        df['price_category'] = pd.qcut(df['sale_price'], q=3, 
                                        labels=['Low', 'Medium', 'High'], 
                                        duplicates='drop')
    
    return df

df = load_data()

# =============================
# Header
# =============================
st.title("🏙️ NYC Real Estate Analysis Dashboard")
st.markdown("**Explore property sales data across all 5 NYC boroughs**")
st.markdown("---")

# =============================
# Sidebar Filters
# =============================
st.sidebar.header("📊 Filters")

# Borough filter
borough = st.sidebar.multiselect(
    "Select Borough(s)",
    options=sorted(df["borough"].unique()),
    default=df["borough"].unique()
)

# Year filter
available_years = sorted(df["sale_year"].unique())
year = st.sidebar.multiselect(
    "Select Year(s)",
    options=available_years,
    default=available_years
)

# Price range filter
price_min = int(df["sale_price"].min())
price_max = int(df["sale_price"].max())

min_price, max_price = st.sidebar.slider(
    "Price Range ($)",
    min_value=price_min,
    max_value=price_max,
    value=(10000, 1000000),
    step=10000,
    format="$%d"
)

# Property size filter
if st.sidebar.checkbox("Filter by Property Size", value=False):
    size_min = int(df["gross_square_feet"].min())
    size_max = int(df["gross_square_feet"].max())
    
    min_size, max_size = st.sidebar.slider(
        "Property Size (sqft)",
        min_value=size_min,
        max_value=min(size_max, 10000),
        value=(500, 5000),
        step=100
    )
    
    filtered_df = df[
        (df["borough"].isin(borough)) &
        (df["sale_year"].isin(year)) &
        (df["sale_price"].between(min_price, max_price)) &
        (df["gross_square_feet"].between(min_size, max_size))
    ]
else:
    filtered_df = df[
        (df["borough"].isin(borough)) &
        (df["sale_year"].isin(year)) &
        (df["sale_price"].between(min_price, max_price))
    ]

# Display filter info
st.sidebar.markdown("---")
st.sidebar.info(f"**{len(filtered_df):,}** properties match your filters")

# =============================
# Key Metrics (KPIs)
# =============================
st.subheader("📈 Key Metrics")

col1, col2, col3, col4, col5 = st.columns(5)

with col1:
    avg_price = filtered_df['sale_price'].mean()
    st.metric(
        label="Average Price",
        value=f"${avg_price:,.0f}"
    )

with col2:
    median_price = filtered_df['sale_price'].median()
    st.metric(
        label="Median Price",
        value=f"${median_price:,.0f}"
    )

with col3:
    avg_sqft_price = filtered_df['price_per_sqft'].mean()
    st.metric(
        label="Avg Price/Sqft",
        value=f"${avg_sqft_price:,.0f}"
    )

with col4:
    avg_size = filtered_df['gross_square_feet'].mean()
    st.metric(
        label="Avg Size",
        value=f"{avg_size:,.0f} sqft"
    )

with col5:
    total_sales = len(filtered_df)
    st.metric(
        label="Total Sales",
        value=f"{total_sales:,}"
    )

st.markdown("---")

# =============================
# Tabs
# =============================
tab1, tab2, tab3, tab4 = st.tabs([
    "📊 Price Analysis",
    "🗺️ Borough Comparison",
    "📈 Trends Over Time",
    "🔍 Property Analysis"
])

# -------------------------
# Tab 1 - Price Analysis
# -------------------------
with tab1:
    st.header("Price Distribution Analysis")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Sale Price Distribution")
        fig1 = px.histogram(
            filtered_df, 
            x="sale_price", 
            nbins=40,
            title="Distribution of Sale Prices",
            labels={'sale_price': 'Sale Price ($)', 'count': 'Number of Properties'},
            color_discrete_sequence=['#1f77b4']
        )
        fig1.update_layout(showlegend=False)
        st.plotly_chart(fig1, use_container_width=True)
        
        # Show statistics
        st.markdown(f"""
        **Statistics:**
        - Mean: ${filtered_df['sale_price'].mean():,.0f}
        - Median: ${filtered_df['sale_price'].median():,.0f}
        - Std Dev: ${filtered_df['sale_price'].std():,.0f}
        """)
    
    with col2:
        st.subheader("Log-Transformed Distribution")
        fig2 = px.histogram(
            filtered_df, 
            x="log_sale_price", 
            nbins=40,
            title="Log-Transformed Sale Price Distribution",
            labels={'log_sale_price': 'Log(Sale Price)', 'count': 'Number of Properties'},
            color_discrete_sequence=['#ff7f0e']
        )
        fig2.update_layout(showlegend=False)
        st.plotly_chart(fig2, use_container_width=True)
        
        st.info("Log transformation helps visualize the distribution when data is skewed.")
    
    # Price per sqft distribution
    st.subheader("Price per Square Foot Distribution")
    fig3 = px.histogram(
        filtered_df, 
        x="price_per_sqft", 
        nbins=50,
        title="Distribution of Price per Square Foot",
        labels={'price_per_sqft': 'Price per Sqft ($)', 'count': 'Number of Properties'},
        color_discrete_sequence=['#2ca02c']
    )
    fig3.update_layout(showlegend=False)
    st.plotly_chart(fig3, use_container_width=True)
    
    # Market segmentation
    st.subheader("Market Segmentation")
    col1, col2 = st.columns([1, 2])
    
    with col1:
        category_counts = filtered_df['price_category'].value_counts()
        fig4 = px.pie(
            values=category_counts.values,
            names=category_counts.index,
            title="Properties by Price Category",
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        st.plotly_chart(fig4, use_container_width=True)
    
    with col2:
        st.markdown("""
        ### Price Categories:
        
        Properties are divided into three equal groups:
        - **Low**: Bottom 33% of prices
        - **Medium**: Middle 33% of prices  
        - **High**: Top 33% of prices
        
        This segmentation helps identify market distribution and opportunities 
        in different price ranges.
        """)

# -------------------------
# Tab 2 - Borough Comparison
# -------------------------
with tab2:
    st.header("Borough-Level Analysis")
    
    # Average price by borough
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Average Sale Price by Borough")
        avg_price_borough = filtered_df.groupby("borough")["sale_price"].mean().reset_index()
        avg_price_borough = avg_price_borough.sort_values("sale_price", ascending=False)
        
        fig5 = px.bar(
            avg_price_borough, 
            x="borough", 
            y="sale_price",
            title="Average Sale Price by Borough",
            labels={'borough': 'Borough', 'sale_price': 'Average Sale Price ($)'},
            text_auto='.2s',
            color='sale_price',
            color_continuous_scale='Blues'
        )
        fig5.update_layout(showlegend=False)
        st.plotly_chart(fig5, use_container_width=True)
    
    with col2:
        st.subheader("Average Price/Sqft by Borough")
        avg_sqft_borough = filtered_df.groupby("borough")["price_per_sqft"].mean().reset_index()
        avg_sqft_borough = avg_sqft_borough.sort_values("price_per_sqft", ascending=False)
        
        fig6 = px.bar(
            avg_sqft_borough, 
            x="borough", 
            y="price_per_sqft",
            title="Average Price per Sqft by Borough",
            labels={'borough': 'Borough', 'price_per_sqft': 'Avg Price/Sqft ($)'},
            text_auto='.2s',
            color='price_per_sqft',
            color_continuous_scale='Oranges'
        )
        fig6.update_layout(showlegend=False)
        st.plotly_chart(fig6, use_container_width=True)
    
    # Box plot comparison
    st.subheader("Price Distribution by Borough")
    fig7 = px.box(
        filtered_df, 
        x="borough", 
        y="sale_price",
        title="Sale Price Distribution Across Boroughs",
        labels={'borough': 'Borough', 'sale_price': 'Sale Price ($)'},
        color='borough',
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig7.update_layout(showlegend=False)
    st.plotly_chart(fig7, use_container_width=True)
    
    # Borough statistics table
    st.subheader("Detailed Borough Statistics")
    borough_stats = filtered_df.groupby('borough').agg({
        'sale_price': ['count', 'mean', 'median', 'std'],
        'price_per_sqft': 'mean',
        'gross_square_feet': 'mean'
    }).round(2)
    borough_stats.columns = ['Count', 'Avg Price', 'Median Price', 'Std Dev', 'Avg $/Sqft', 'Avg Size (sqft)']
    st.dataframe(borough_stats, use_container_width=True)

# -------------------------
# Tab 3 - Trends Over Time
# -------------------------
with tab3:
    st.header("Temporal Trends Analysis")
    
    # Price trend over time
    st.subheader("Average Sale Price Over Time")
    yearly_avg = filtered_df.groupby("sale_year")["sale_price"].mean().reset_index()
    
    fig8 = px.line(
        yearly_avg, 
        x="sale_year", 
        y="sale_price",
        title="Average Sale Price Trend",
        labels={'sale_year': 'Year', 'sale_price': 'Average Sale Price ($)'},
        markers=True,
        color_discrete_sequence=['#d62728']
    )
    fig8.update_traces(line=dict(width=3), marker=dict(size=10))
    st.plotly_chart(fig8, use_container_width=True)
    
    # Year-over-year growth
    if len(yearly_avg) > 1:
        yearly_avg['yoy_growth'] = yearly_avg['sale_price'].pct_change() * 100
        st.markdown("**Year-over-Year Growth Rate:**")
        st.dataframe(
            yearly_avg[['sale_year', 'sale_price', 'yoy_growth']].dropna(),
            use_container_width=True
        )
    
    # Sales volume over time
    st.subheader("Sales Volume Over Time")
    yearly_volume = filtered_df.groupby("sale_year").size().reset_index(name='count')
    
    fig9 = px.bar(
        yearly_volume,
        x="sale_year",
        y="count",
        title="Number of Sales by Year",
        labels={'sale_year': 'Year', 'count': 'Number of Sales'},
        text_auto=True,
        color='count',
        color_continuous_scale='Greens'
    )
    st.plotly_chart(fig9, use_container_width=True)
    
    # Trend by borough
    st.subheader("Price Trends by Borough")
    yearly_borough = filtered_df.groupby(["sale_year", "borough"])["sale_price"].mean().reset_index()
    
    fig10 = px.line(
        yearly_borough,
        x="sale_year",
        y="sale_price",
        color="borough",
        title="Average Price Trend by Borough",
        labels={'sale_year': 'Year', 'sale_price': 'Avg Price ($)', 'borough': 'Borough'},
        markers=True
    )
    st.plotly_chart(fig10, use_container_width=True)

# -------------------------
# Tab 4 - Property Analysis
# -------------------------
with tab4:
    st.header("Property Characteristics Analysis")
    
    # Property size vs price
    st.subheader("Property Size vs Sale Price")
    
    # Sample for better performance
    sample_size = min(5000, len(filtered_df))
    df_sample = filtered_df.sample(n=sample_size, random_state=42)
    
    fig11 = px.scatter(
        df_sample,
        x="gross_square_feet",
        y="sale_price",
        color="borough",
        title=f"Property Size vs Price (Sample of {sample_size:,} properties)",
        labels={'gross_square_feet': 'Size (sqft)', 'sale_price': 'Price ($)', 'borough': 'Borough'},
        opacity=0.6,
        hover_data=['price_per_sqft']
    )
    st.plotly_chart(fig11, use_container_width=True)
    
    correlation = filtered_df['sale_price'].corr(filtered_df['gross_square_feet'])
    st.info(f"**Correlation between size and price:** {correlation:.3f}")
    
    # Property age vs price
    st.subheader("Property Age vs Sale Price")
    
    fig12 = px.scatter(
        df_sample,
        x="property_age",
        y="sale_price",
        color="borough",
        title="Property Age vs Price",
        labels={'property_age': 'Age (years)', 'sale_price': 'Price ($)', 'borough': 'Borough'},
        opacity=0.6
    )
    st.plotly_chart(fig12, use_container_width=True)
    
    age_corr = filtered_df['property_age'].corr(filtered_df['sale_price'])
    st.info(f"**Correlation between age and price:** {age_corr:.3f}")
    
    # Units vs price
    st.subheader("Number of Units vs Sale Price")
    
    df_units = filtered_df[filtered_df['total_units'] <= 50]
    df_sample_units = df_units.sample(n=min(3000, len(df_units)), random_state=42)
    
    fig13 = px.scatter(
        df_sample_units,
        x="total_units",
        y="sale_price",
        color="price_per_sqft",
        title="Total Units vs Price",
        labels={'total_units': 'Number of Units', 'sale_price': 'Price ($)', 
                'price_per_sqft': 'Price/Sqft ($)'},
        opacity=0.6,
        color_continuous_scale='Viridis'
    )
    st.plotly_chart(fig13, use_container_width=True)
    
    # Correlation heatmap
    st.subheader("Correlation Heatmap")
    
    numeric_cols = ['sale_price', 'gross_square_feet', 'land_square_feet', 
                    'total_units', 'property_age', 'price_per_sqft']
    corr_matrix = filtered_df[numeric_cols].corr()
    
    fig14 = px.imshow(
        corr_matrix,
        text_auto='.2f',
        title="Correlation Matrix of Key Variables",
        color_continuous_scale='RdBu_r',
        aspect='auto'
    )
    st.plotly_chart(fig14, use_container_width=True)
    
    st.markdown("""
    **Interpretation:**
    - Values close to 1: Strong positive correlation
    - Values close to -1: Strong negative correlation
    - Values close to 0: Weak or no correlation
    """)

# =============================
# Footer
# =============================
st.markdown("---")
st.markdown("""
<div style='text-align: center'>
    <p><strong>NYC Real Estate Analysis Dashboard</strong></p>
    <p>Data source: NYC Department of Finance Rolling Sales Data</p>
</div>
""", unsafe_allow_html=True)


Overwriting app.py


In [33]:
!streamlit run app.py

^C


## 10. Conclusions

This analysis of NYC real estate data revealed several important insights:

1. **Location matters most**: Borough is the strongest predictor of price
2. **Size affects price**: Larger properties are more expensive
3. **Age less important**: Old and new properties both have value
4. **Market growing**: Overall upward price trend
5. **Wide price range**: Properties vary greatly in value

These findings can help buyers, sellers, and investors make better decisions in the NYC real estate market.